In [1]:
import pandas as pd
from api import get_api
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

### Cluster의 vrp 정보를 가져온다.

In [2]:
solution = pd.read_csv('solution_cluster_info.csv')

In [3]:
solution

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,50,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,50,1,41.0,23.0,39.0,50.0,NaN,NaN,NaN,NaN,NaN
2,2,50,15,20.0,22.0,6.0,43.0,35.0,50.0,NaN,NaN,NaN
3,3,50,18,40.0,4.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,50,19,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,50,45,49.0,26.0,38.0,0.0,27.0,37.0,8.0,50.0,NaN
6,6,50,31,14.0,7.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
7,7,50,46,17.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,50,32,13.0,9.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
9,9,50,5,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cluster 정보를 나타낸다.

In [4]:
spot = pd.read_csv('clusterinfo.csv')
spot

,Unnamed: 0,lat,lng,cluster
0,1592,35.218610,126.844707,0
1,73,35.219804,126.845734,0
2,1537,35.219165,126.845790,0
3,1705,35.219764,126.846482,0
4,346,35.219443,126.845734,0
...,...,...,...,...
2041,1188,35.220024,126.836280,48
2042,27,35.216719,126.838153,48
2043,466,35.218801,126.838391,48
2044,1786,35.231681,126.851529,49


### Cluster된 정보를 가져오는 함수

In [5]:
def find(cNum):
    cNum = cNum[2:]
    result = pd.DataFrame()
    for i in cNum:
        if(i==50):
            break
        temp = spot[spot['cluster']==i]
        result = pd.concat([result, temp],ignore_index=True,axis=0)
        result = result[['lat','lng']]
    return result

In [6]:
solutionList = solution.values.tolist()
arrive = {'lat': 35.229546, 'lng' : 126.847516}

In [7]:
def calcdistance(A,B):
    return abs(A['lat']-B['lat']) + abs(A['lng']-B['lng'])

In [8]:
def calc_distance(originX,originY,destinationX, destinationY):
    # origin and destination should be coordinate
    url = 'https://apis-navi.kakaomobility.com/v1/directions'
    params = {'origin': str(originX) +','+str(originY),
              'destination': str(destinationX) + ','+str(destinationY)}
    header = {'Authorization': 'KakaoAK ' + get_api()}

    route = requests.get(url, params=params, headers=header).json()
#     print(route)

    if route['routes'][0]['result_code'] == 104:
        return 0
    else:
        return(route['routes'][0]['summary']['duration'])

In [9]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/60)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance/60))

In [10]:
print('start')

start


In [ ]:
answer = []


for i in solutionList:
    newDF = pd.DataFrame(find(i))
    if(len(newDF) <= 0):
        continue
    newDF.append(arrive,ignore_index=True)
    result = []
    for j in range(len(newDF)-1):
        temp = []
        for k in range(len(newDF)):
            distance = calcdistance(newDF.iloc[j],newDF.iloc[k]) * 10000
            temp.append(distance)
        result.append(temp)
    result.append([0]*len(newDF))
    
    def create_data_model():
        data = {}
        data['distance_matrix'] = result
        data['num_vehicles'] = 3
        data['depot'] = len(result)-1
        return data
    
    # Instantiate the data problem.

    data = create_data_model()
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node] 
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        100000,  # no slack
        30000000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    def print_solution_only_num():
        output = []
        for vehicle_id in range(data['num_vehicles']):
            Output = []
            index = routing.Start(vehicle_id)
            while not routing.IsEnd(index):
                Output.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            Output.append(manager.IndexToNode(index))  
            output.append(Output)
        return output
    
    if solution:
        print_solution(data, manager, routing, solution)
        sol = print_solution_only_num()
        answer.append(sol)
        print(sol)
    else:
        print('No solution found !')

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2578/3051611209.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)


Objective: 0
Route for vehicle 0:
 217 ->  162 ->  192 ->  112 ->  212 ->  143 ->  166 ->  204 ->  55 -> 217
Distance of the route: 0.0m

Route for vehicle 1:
 217 ->  41 ->  154 ->  126 ->  60 ->  129 ->  151 -> 217
Distance of the route: 0.0m

Route for vehicle 2:
 217 ->  216 ->  215 ->  214 ->  210 ->  207 ->  205 ->  202 ->  200 ->  199 ->  196 ->  193 ->  191 ->  190 ->  189 ->  187 ->  185 ->  184 ->  183 ->  182 ->  181 ->  180 ->  175 ->  172 ->  170 ->  173 ->  213 ->  206 ->  203 ->  197 ->  195 ->  177 ->  176 ->  174 ->  167 ->  165 ->  171 ->  211 ->  209 ->  201 ->  198 ->  194 ->  188 ->  179 ->  178 ->  169 ->  168 ->  208 ->  186 ->  148 ->  117 ->  164 ->  160 ->  150 ->  139 ->  138 ->  137 ->  134 ->  132 ->  122 ->  152 ->  145 ->  144 ->  140 ->  127 ->  123 ->  120 ->  119 ->  118 ->  161 ->  141 ->  131 ->  130 ->  163 ->  158 ->  153 ->  149 ->  159 ->  155 ->  146 ->  121 ->  133 ->  157 ->  156 ->  135 ->  128 ->  125 ->  124 ->  147 ->  136 ->  142 ->  100 

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2578/3051611209.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)


In [ ]:
answerDF = pd.DataFrame(answer)

In [ ]:
answerDF.to_csv('allDF.csv',index=False)